In [20]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

### HSCODE

In [28]:
hscode = pd.read_csv('./data/HS Code.csv',encoding='euc-kr')
hscode

,HS코드,MTI코드
0,101900000,21190
1,104101000,21190
2,104109000,21190
3,104201000,21190
4,104209000,21190
...,...,...
16111,8703901000,999000
16112,8703902000,999000
16113,8703904000,999000
16114,8703905000,999000


### 표준품명

In [13]:
# standard_item_name = pd.read_csv('../data/표준품명(2023).csv')
standard_item_name = pd.read_csv('./DATA/관세청_표준품명.csv')
standard_item_name.head()

,연번,품명,HS부호,품명번호,표준품명_한글,표준품명_영문,규격번호,규격순번,필수규격_한글,필수규격_영문,규격값,세부분류내용,비고,시작일자,만료일자,용도구분
0,1,노새,101900000,1,노새,Mule,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-09-23,2023-12-31,기타세번
1,1,버새,101900000,2,버새,Hinny,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-09-23,2023-12-31,기타세번
2,1,기타,101900000,3,기타,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-09-23,2023-12-31,기타세번
3,2,번식용,102310000,1,번식용,Purebred breeding animals,1.0,1.0,NaN,NaN,1,젖소 및 육우,NaN,2017-02-24,2023-12-31,FTA세율
4,2,번식용,102310000,1,번식용,Purebred breeding animals,1.0,2.0,NaN,NaN,2,기타,NaN,2017-02-24,2023-12-31,FTA세율


In [14]:
code = list(standard_item_name['HS부호'].values)
hscodes = set(code)
len(hscodes)

925

### 관체청_품목별 수출입실적(최근 3개년)(전체품목)

In [167]:
import itertools
from tqdm import tqdm
import time
# serviceKey = "fGflzKnW7C1SsvwQEI%2BM6gYe0bhKAafPztsLdMa1oc7lBOXSyOZPaJzuy2ehIBI%2FoMQcPy8oqSoA8jgtKJYHOw%3D%3D"
serviceKey = "eOCWH%2FCPN7A3KO79lNiCypt%2BHQYgfm6wZz1oMjycwpy0%2FVso2V8B15%2BbF%2BW4C2ebWfLahhBwe0YeFMjSA4qbzA%3D%3D"

In [168]:
startDate = "202001"
endDate = "202012"
urls = []
for hscode in hscodes:
    url = f"https://apis.data.go.kr/1220000/Itemtrade/getItemtradeList?serviceKey={serviceKey}&strtYymm={startDate}&endYymm={endDate}&hsSgn={hscode}"
    urls.append(url)

In [169]:
te_li = []
for url in tqdm(urls):
    result = urlopen(url)
    house = BeautifulSoup(result, "lxml-xml")
    te = house.find_all("item")
    te_li.append(te)

te = list(itertools.chain(*te_li))

100%|████████████████████████████████████████████████████████████████████████████████| 925/925 [02:18<00:00,  6.66it/s]


In [170]:
# 수출입통계정보의 기간, 품목명, HS코드, 수입/수출 중량 및 금액, 무역수지
datas = []

for i in range(len(te)):
    balPayments = te[i].balPayments.string
    expDlr = te[i].expDlr.string
    expWgt = te[i].expWgt.string
    hsCode = te[i].hsCode.string
    impDlr = te[i].impDlr.string
    impWgt = te[i].impWgt.string
    statKor = te[i].statKor.string
    year = te[i].year.string
    
    data = [balPayments, expDlr, expWgt, hsCode, impDlr, impWgt, statKor, year]
    datas.append(data)

In [171]:
df = pd.DataFrame(datas, columns=["무역수지", "수출금액(달러)", "수출중량(KG)",
                                  "HS코드", "수입금액(달러)", "수입중량(KG)",
                                  "품목명", "기간"])
df["무역수지"] = df["무역수지"].astype(int)
df["수출금액(달러)"] = df["수출금액(달러)"].astype(int)
df["수출중량(KG)"] = df["수출중량(KG)"].astype(int)
df["수입금액(달러)"] = df["수입금액(달러)"].astype(int)
import_export_item_2020 = df 
import_export_item_2020

,무역수지,수출금액(달러),수출중량(KG),HS코드,수입금액(달러),수입중량(KG),품목명,기간
0,-4246434,0,0,6907221000,4246434,18089657,수분흡수계수가 중량기준으로 100분의 0.5 초과 100분의 3 이하인 것,2020.01
1,-1541557,9318,12800,6907221000,1550875,5993751,수분흡수계수가 중량기준으로 100분의 0.5 초과 100분의 3 이하인 것,2020.02
2,-2820885,0,0,6907221000,2820885,12133282,수분흡수계수가 중량기준으로 100분의 0.5 초과 100분의 3 이하인 것,2020.03
3,-4709647,0,0,6907221000,4709647,20951289,수분흡수계수가 중량기준으로 100분의 0.5 초과 100분의 3 이하인 것,2020.04
4,-4612499,0,0,6907221000,4612499,20232602,수분흡수계수가 중량기준으로 100분의 0.5 초과 100분의 3 이하인 것,2020.05
5,-4929222,0,0,6907221000,4929222,21956294,수분흡수계수가 중량기준으로 100분의 0.5 초과 100분의 3 이하인 것,2020.06
6,-4183414,0,0,6907221000,4183414,17671234,수분흡수계수가 중량기준으로 100분의 0.5 초과 100분의 3 이하인 것,2020.07
7,-3762934,4956,9690,6907221000,3767890,17267067,수분흡수계수가 중량기준으로 100분의 0.5 초과 100분의 3 이하인 것,2020.08
8,-3346166,0,0,6907221000,3346166,15055327,수분흡수계수가 중량기준으로 100분의 0.5 초과 100분의 3 이하인 것,2020.09
9,-4714623,0,0,6907221000,4714623,21625755,수분흡수계수가 중량기준으로 100분의 0.5 초과 100분의 3 이하인 것,2020.10


In [36]:
# import_export_item_2022.to_csv('./data/import_export_item_2022(월별).csv',index=False,encoding='euc-kr')

In [153]:
# import_export_item_2021.to_csv('./data/import_export_item_2021(월별).csv',index=False,encoding='euc-kr')

In [172]:
import_export_item_2020.to_csv('./data/import_export_item_2020(월별).csv',index=False,encoding='euc-kr')

### 관세청_품목별 수출입실적_2017~2022(전체 품목)

In [117]:
# urls_in = []
# urls_out = []
# # years = [2017,2018,2019,2020,2021,2022]
# years = [2021,2022]
# for year in years:
#     startdate = f"{year}01"
#     enddate = f"{year}12"
#     for hscode in hscodes:
#         url = f"https://apis.data.go.kr/1220000/Itemtrade/getItemtradeList?serviceKey={serviceKey}&strtYymm={startdate}&endYymm={enddate}&hsSgn={hscode}"
#         urls_in.append(url)
#     urls_out.extend(urls_in)

In [154]:
urls_out = []
years = [2021,2022]
for year in years:
    urls_in = [] 
    startdate = f"{year}01"
    enddate = f"{year}12"
    for hscode in hscodes:
        url = f"https://apis.data.go.kr/1220000/Itemtrade/getItemtradeList?serviceKey={serviceKey}&strtYymm={startdate}&endYymm={enddate}&hsSgn={hscode}"
        urls_in.append(url)
    urls_out.extend(urls_in)

In [157]:
te_li = []
for url in tqdm(urls_out):
    result = urlopen(url)
    house = BeautifulSoup(result, "lxml-xml")
    te = house.find_all("item")
    te_li.append(te)

te = list(itertools.chain(*te_li))

100%|██████████████████████████████████████████████████████████████████████████████| 1850/1850 [04:38<00:00,  6.63it/s]


In [128]:
# 수출입통계정보의 기간, 품목명, HS코드, 수입/수출 중량 및 금액, 무역수지
datas = []

for i in range(len(te)):
    balPayments = te[i].balPayments.string
    expDlr = te[i].expDlr.string
    expWgt = te[i].expWgt.string
    hsCode = te[i].hsCode.string
    impDlr = te[i].impDlr.string
    impWgt = te[i].impWgt.string
    statKor = te[i].statKor.string
    year = te[i].year.string
    
    data = [balPayments, expDlr, expWgt, hsCode, impDlr, impWgt, statKor, year]
    datas.append(data)

In [135]:
df = pd.DataFrame(datas, columns=["무역수지", "수출금액(달러)", "수출중량(KG)",
                                  "HS코드", "수입금액(달러)", "수입중량(KG)",
                                  "품목명", "기간"])
df["무역수지"] = df["무역수지"].astype(int)
df["수출금액(달러)"] = df["수출금액(달러)"].astype(int)
df["수출중량(KG)"] = df["수출중량(KG)"].astype(int)
df["수입금액(달러)"] = df["수입금액(달러)"].astype(int)
import_export_item_2017_2022 = df 
import_export_item_2017_2022

,무역수지,수출금액(달러),수출중량(KG),HS코드,수입금액(달러),수입중량(KG),품목명,기간
0,-449669,912130,55219,2101209090,1361799,46107,기타,2021.01
1,-571098,701993,49424,2101209090,1273091,64103,기타,2021.02
2,-862934,1172804,32842,2101209090,2035738,82189,기타,2021.03
3,-480414,1067669,90777,2101209090,1548083,70896,기타,2021.04
4,-648792,621627,74504,2101209090,1270419,50956,기타,2021.05
...,...,...,...,...,...,...,...,...
1544,-2709,0,0,8504501010,2709,7,"자동자료처리기계와 그 단위기기(제8471호, 제8443.31호, 제8443.32호,...",2021.07
1545,-5799,0,0,8504501010,5799,65,"자동자료처리기계와 그 단위기기(제8471호, 제8443.31호, 제8443.32호,...",2021.09
1546,-2324,0,0,8504501010,2324,8,"자동자료처리기계와 그 단위기기(제8471호, 제8443.31호, 제8443.32호,...",2021.11
1547,41180,43288,6440,8504501010,2108,15,"자동자료처리기계와 그 단위기기(제8471호, 제8443.31호, 제8443.32호,...",2021.12


In [152]:
df = pd.DataFrame(datas, columns=["무역수지", "수출금액(달러)", "수출중량(KG)",
                                  "HS코드", "수입금액(달러)", "수입중량(KG)",
                                  "품목명", "기간"])
df["무역수지"] = df["무역수지"].astype(int)
df["수출금액(달러)"] = df["수출금액(달러)"].astype(int)
df["수출중량(KG)"] = df["수출중량(KG)"].astype(int)
df["수입금액(달러)"] = df["수입금액(달러)"].astype(int)
all_import_export_item_2021 = df
all_import_export_item_2021

,무역수지,수출금액(달러),수출중량(KG),HS코드,수입금액(달러),수입중량(KG),품목명,기간
0,-449669,912130,55219,2101209090,1361799,46107,기타,2021.01
1,-571098,701993,49424,2101209090,1273091,64103,기타,2021.02
2,-862934,1172804,32842,2101209090,2035738,82189,기타,2021.03
3,-480414,1067669,90777,2101209090,1548083,70896,기타,2021.04
4,-648792,621627,74504,2101209090,1270419,50956,기타,2021.05
...,...,...,...,...,...,...,...,...
1544,-2709,0,0,8504501010,2709,7,"자동자료처리기계와 그 단위기기(제8471호, 제8443.31호, 제8443.32호,...",2021.07
1545,-5799,0,0,8504501010,5799,65,"자동자료처리기계와 그 단위기기(제8471호, 제8443.31호, 제8443.32호,...",2021.09
1546,-2324,0,0,8504501010,2324,8,"자동자료처리기계와 그 단위기기(제8471호, 제8443.31호, 제8443.32호,...",2021.11
1547,41180,43288,6440,8504501010,2108,15,"자동자료처리기계와 그 단위기기(제8471호, 제8443.31호, 제8443.32호,...",2021.12


In [ ]:
all_import_export_item_2018~2022.to_csv('./data/all_import_export_item_2017_2022.csv',index=False,encoding='euc-kr')

### 관세청_품목별 국가별 수출입실적

In [ ]:
import pandas as pd
from tqdm import tqdm
from urllib.request import urlopen
from bs4 import BeautifulSoup
import itertools

def get_trade_data(cntyCd, years, hscodes, serviceKey):
    urls_in = []
    urls_out = []
    
    for year in years:
        startdate = f"{year}01"
        enddate = f"{year}12"
        for hscode in hscodes:
            url = f"https://apis.data.go.kr/1220000/Itemtrade/getItemtradeList?serviceKey={serviceKey}&strtYymm={startdate}&endYymm={enddate}&hsSgn={hscode}&cntyCd={cntyCd}"
            urls_in.append(url)
        urls_out.extend(urls_in)
        
    te_li = []
    for url in tqdm(urls_out):
        result = urlopen(url)
        house = BeautifulSoup(result, "lxml-xml")
        te = house.find_all("item")
        te_li.append(te)
    
    te = list(itertools.chain(*te_li))
    
    datas = []
    
    for i in range(len(te)):
        balPayments = te[i].balPayments.string
        expDlr = te[i].expDlr.string
        expWgt = te[i].expWgt.string
        hsCode = te[i].hsCode.string
        impDlr = te[i].impDlr.string
        impWgt = te[i].impWgt.string
        statKor = te[i].statKor.string
        year = te[i].year.string
        
        data = [balPayments, expDlr, expWgt, hsCode, impDlr, impWgt, statKor, year]
        datas.append(data)
        
    df = pd.DataFrame(datas, columns=["무역수지", "수출금액(달러)", "수출중량(KG)",
                                      "HS코드", "수입금액(달러)", "수입중량(KG)",
                                      "품목명", "기간"])
    df["무역수지"] = df["무역수지"].astype(int)
    df["수출금액(달러)"] = df["수출금액(달러)"].astype(int)
    df["수출중량(KG)"] = df["수출중량(KG)"].astype(int)
    df["수입금액(달러)"] = df["수입금액(달러)"].astype(int)
    
    return df

### 관세청_품목별 국가별 수출입실적_2018~2022(품목1개)

In [41]:
serviceKey = "eOCWH%2FCPN7A3KO79lNiCypt%2BHQYgfm6wZz1oMjycwpy0%2FVso2V8B15%2BbF%2BW4C2ebWfLahhBwe0YeFMjSA4qbzA%3D%3D"
itemCode = "2309102000"
#상위 10개 국가 선택하기
cntyCd = 'US'

urls = []
years = [2018,2019,2020,2021,2022]
for year in years:
    startdate = f"{year}01"
    enddate = f"{year}12"
    url = f"http://apis.data.go.kr/1220000/nitemtrade/getNitemtradeList?serviceKey={serviceKey}&strtYymm={startdate}&endYymm={enddate}&hsSgn={itemCode}&cntyCd={cntyCd}"
    urls.append(url)

In [ ]:
import itertools

te_li = []
for url in urls:
    result = urlopen(url)
    house = BeautifulSoup(result, "lxml-xml")
    te = house.find_all("item")
    te_li.append(te)

te = list(itertools.chain(*te_li))

In [46]:
datas = []

for i in range(len(te)):
    year = te[i].year.string
    statCdCntnKor1 = te[i].statCdCntnKor1.string
    statCd = te[i].statCd.string
    statKor = te[i].statKor.string
    hsCode = te[i].hsCd.string
    expWgt = te[i].expWgt.string
    expDlr = te[i].expDlr.string
    impWgt = te[i].impWgt.string
    impDlr = te[i].impDlr.string
    balPayments = te[i].balPayments.string
    
    data = [year,statCdCntnKor1,statCd,statKor,hsCode,expWgt,expDlr,impWgt,impDlr,balPayments]
    datas.append(data)

In [49]:
df = pd.DataFrame(datas, columns=["기간", "국가명", "국가코드","품목명","HS코드",
                                 "수출중량(kg)","수출금액(달러)","수입중량(kg)","수입금액(달러)",
                                 "무역수지(달러)"])

In [56]:
df["수출금액(달러)"] = df["수출금액(달러)"].astype(int)
df["수출중량(kg)"] = df["수출중량(kg)"].astype(int)
df["수입금액(달러)"] = df["수입금액(달러)"].astype(int)
df["수입중량(kg)"] = df["수입중량(kg)"].astype(int)
df["무역수지(달러)"] = df["무역수지(달러)"].astype(int)
country_import_export_item = df
country_import_export_item

,기간,국가명,국가코드,품목명,HS코드,수출중량(kg),수출금액(달러),수입중량(kg),수입금액(달러),무역수지(달러)
0,총계,-,-,-,-,165,3300,6048099,14360765,-14357465
1,2015.01,미국,US,고양이 사료,2309102000,0,0,661446,1552895,-1552895
2,2015.02,미국,US,고양이 사료,2309102000,0,0,324989,696601,-696601
3,2015.03,미국,US,고양이 사료,2309102000,0,0,377351,821596,-821596
4,2015.04,미국,US,고양이 사료,2309102000,165,3300,676011,1670842,-1667542
...,...,...,...,...,...,...,...,...,...,...
99,2022.08,미국,US,고양이 사료,2309102000,153,20505,952647,3388184,-3367679
100,2022.09,미국,US,고양이 사료,2309102000,1689,16531,732317,2709680,-2693149
101,2022.10,미국,US,고양이 사료,2309102000,7,260,997909,3237092,-3236832
102,2022.11,미국,US,고양이 사료,2309102000,72,11529,1034722,3709135,-3697606
